# Análise Exploratória de Dados - Posição de Clientes MMZR

Este notebook realiza uma análise exploratória inicial dos dados de posição de clientes. O objetivo é entender a estrutura do arquivo Excel fornecido, identificar colunas-chave e extrair insights preliminares para a consolidação das carteiras de investimento.

In [ ]:
import pandas as pd
import numpy as np
import os

# OBRIGATÓRIO: Verificar se arquivo existe
caminho_arquivo = './dados/posicao_de_clientes.xlsx'
if not os.path.exists(caminho_arquivo):
    print(f"ERRO: Arquivo não encontrado: {caminho_arquivo}")
    print("Verifique se o arquivo está na pasta correta.")
else:
    try:
        df = pd.read_excel(caminho_arquivo)
        print(f"✅ Dataset carregado com sucesso: {df.shape[0]} linhas, {df.shape[1]} colunas")
    except Exception as e:
        print(f"ERRO ao carregar arquivo: {e}")
        df = None

## 1. Estrutura Inicial dos Dados

A primeira etapa é carregar os dados e realizar uma inspeção básica da sua estrutura, incluindo a identificação das colunas disponíveis, o formato geral do dataset e os tipos de dados de cada coluna. Também visualizamos as primeiras linhas para ter uma ideia do conteúdo.

In [ ]:
# OBRIGATÓRIO: Verificar se dados foram carregados
if df is not None:
    print("=== ESTRUTURA DOS DADOS ===")
    print("Colunas disponíveis:")
    for i, col in enumerate(df.columns):
        print(f"  {i+1}. {col}")
    
    print(f"
Formato dos dados: {df.shape}")
    print(f"Tipos de dados:
{df.dtypes}")
    
    print("
=== PRIMEIRAS 3 LINHAS ===")
    display(df.head(3))
    
    # IDENTIFICAR possíveis colunas de cliente, valor, classe de ativo
    colunas_possiveis = {
        'cliente': [col for col in df.columns if any(term in col.lower() for term in ['cliente', 'client', 'nome'])],
        'valor': [col for col in df.columns if any(term in col.lower() for term in ['valor', 'pl', 'patrimonio', 'saldo', 'montante'])],
        'classe': [col for col in df.columns if any(term in col.lower() for term in ['classe', 'ativo', 'estrategia', 'produto'])]
    }
    
    print("
=== COLUNAS IDENTIFICADAS ===")
    for tipo, colunas in colunas_possiveis.items():
        print(f"{tipo.upper()}: {colunas}")
else:
    print("❌ Não é possível analisar - dados não carregados")

## 2. Identificação de Clientes

Analisamos a coluna de clientes para entender quantos clientes únicos existem e como os registros estão distribuídos entre eles.

In [ ]:
if df is not None:
    # ASSUMIR que primeira coluna identificada como 'cliente' é a correta
    # Se não encontrou, usar primeira coluna que contenha texto
    coluna_cliente = None
    if colunas_possiveis['cliente']:
        coluna_cliente = colunas_possiveis['cliente'][0]
    else:
        # Buscar primeira coluna com dados tipo string/object
        for col in df.columns:
            if df[col].dtype == 'object':
                coluna_cliente = col
                break
    
    if coluna_cliente:
        print(f"=== ANÁLISE DE CLIENTES (usando coluna: {coluna_cliente}) ===")
        print(f"Total de registros: {len(df)}")
        print(f"Clientes únicos: {df[coluna_cliente].nunique()}")
        print(f"Registros por cliente (média): {len(df) / df[coluna_cliente].nunique():.1f}")
        
        # Mostrar alguns exemplos de clientes
        print(f"
Exemplos de clientes:")
        clientes_unicos = df[coluna_cliente].unique()
        for i, cliente in enumerate(clientes_unicos[:5]):
            print(f"  {i+1}. {cliente}")
        
        if len(clientes_unicos) > 5:
            print(f"  ... e mais {len(clientes_unicos) - 5} clientes")
    else:
        print("❌ Não foi possível identificar coluna de clientes")
        print("Colunas disponíveis:", df.columns.tolist())
else:
    print("❌ Não é possível analisar - dados não carregados")

## 3. Análise de Classes de Ativo e Classificação Local/Offshore

Identificamos as diferentes classes de ativo presentes nos dados e tentamos classificá-las automaticamente como 'LOCAL', 'OFFSHORE' ou 'INDEFINIDO' com base em termos-chave.

In [ ]:
if df is not None and coluna_cliente:
    # Identificar coluna de classe de ativo
    coluna_classe = None
    if colunas_possiveis['classe']:
        coluna_classe = colunas_possiveis['classe'][0]
    else:
        print("❌ Não foi possível identificar coluna de classes automaticamente")
        print("Colunas disponíveis:", df.columns.tolist())
        
    if coluna_classe:
        print(f"=== ANÁLISE DE CLASSES DE ATIVO (usando coluna: {coluna_classe}) ===")
        classes_unicas = df[coluna_classe].value_counts()
        print(f"Total de classes diferentes: {len(classes_unicas)}")
        print("
Distribuição de classes:")
        print(classes_unicas)
        
        # LÓGICA PARA SEPARAR LOCAL VS OFFSHORE
        print("
=== IDENTIFICAÇÃO LOCAL VS OFFSHORE ===")
        
        # Termos que indicam carteira LOCAL (Brasil)
        termos_local = ['pós', 'pos', 'inflação', 'inflacao', 'pré', 'pre', 'bancário', 'bancario', 
                       'cdi', 'ipca', 'selic', 'brasil', 'br', 'local']
        
        # Termos que indicam carteira OFFSHORE (Internacional)
        termos_offshore = ['equity', 'equities', 'fixed income', 'bond', 'international', 
                          'global', 'usd', 'eur', 'offshore', 'exterior']
        
        # Classificar cada classe
        classificacao = {}
        for classe in classes_unicas.index:
            classe_lower = str(classe).lower()
            
            # Verificar se contém termos locais
            is_local = any(termo in classe_lower for termo in termos_local)
            # Verificar se contém termos offshore
            is_offshore = any(termo in classe_lower for termo in termos_offshore)
            
            if is_local and not is_offshore:
                classificacao[classe] = 'LOCAL'
            elif is_offshore and not is_local:
                classificacao[classe] = 'OFFSHORE'
            else:
                classificacao[classe] = 'INDEFINIDO'
        
        # Mostrar classificação
        print("
Classificação automática:")
        for classe, tipo in classificacao.items():
            print(f"  {tipo}: {classe}")
        
        # Estatísticas da classificação
        tipos_count = {}
        for tipo in classificacao.values():
            tipos_count[tipo] = tipos_count.get(tipo, 0) + 1
        
        print(f"
Resumo da classificação:")
        for tipo, count in tipos_count.items():
            print(f"  {tipo}: {count} classes")
            
    else:
        print("❌ Análise de classes não pode prosseguir sem identificar a coluna correta")
else:
    print("❌ Não é possível analisar - dados não carregados ou coluna de cliente não identificada")

## 4. Análise de Carteiras Mistas

Com base na classificação Local/Offshore, identificamos clientes que possuem tanto ativos locais quanto offshore em suas carteiras.

In [ ]:
if df is not None and coluna_cliente and coluna_classe and 'classificacao' in locals() and classificacao:
    # Criar coluna de tipo de carteira no dataframe
    df['tipo_carteira'] = df[coluna_classe].map(classificacao)
    
    print("=== ANÁLISE DE CARTEIRAS MISTAS ===")
    
    # Agrupar por cliente e verificar tipos de carteira
    clientes_tipos = df.groupby(coluna_cliente)['tipo_carteira'].unique()
    
    # Identificar clientes com carteiras mistas (local E offshore)
    clientes_mistas = []
    clientes_so_local = []
    clientes_so_offshore = []
    clientes_indefinido = []
    
    for cliente, tipos in clientes_tipos.items():
        tipos_set = set(tipos)
        
        if 'LOCAL' in tipos_set and 'OFFSHORE' in tipos_set:
            clientes_mistas.append(cliente)
        elif 'LOCAL' in tipos_set and 'OFFSHORE' not in tipos_set:
            clientes_so_local.append(cliente)
        elif 'OFFSHORE' in tipos_set and 'LOCAL' not in tipos_set:
            clientes_so_offshore.append(cliente)
        else:
            clientes_indefinido.append(cliente)
    
    print(f"Clientes com carteiras MISTAS (local + offshore): {len(clientes_mistas)}")
    print(f"Clientes APENAS locais: {len(clientes_so_local)}")
    print(f"Clientes APENAS offshore: {len(clientes_so_offshore)}")
    print(f"Clientes indefinidos: {len(clientes_indefinido)}")
    
    # Mostrar exemplos de clientes com carteiras mistas
    if clientes_mistas:
        print(f"
Exemplos de clientes com carteiras mistas:")
        for i, cliente in enumerate(clientes_mistas[:3]):
            print(f"  {i+1}. {cliente}")
            cliente_data = df[df[coluna_cliente] == cliente]
            for _, row in cliente_data.iterrows():
                print(f"     {row[coluna_classe]} ({row['tipo_carteira']})")
        
        if len(clientes_mistas) > 3:
            print(f"  ... e mais {len(clientes_mistas) - 3} clientes com carteiras mistas")
    else:
        print("⚠️ Nenhum cliente com carteiras mistas identificado")
        
else:
    print("❌ Não é possível analisar carteiras mistas - dados incompletos")

## 5. Identificação e Validação da Coluna de Valores Monetários

Esta seção foca em encontrar a coluna que representa os valores monetários das posições e realizar uma validação básica (tipo de dado, valores nulos, mínimo, máximo, etc.).

In [ ]:
if df is not None:
    # Identificar coluna de valores
    coluna_valor = None
    if colunas_possiveis['valor']:
        coluna_valor = colunas_possiveis['valor'][0]
    else:
        # Buscar colunas numéricas que possam ser valores
        colunas_numericas = df.select_dtypes(include=[np.number]).columns
        if len(colunas_numericas) > 0:
            coluna_valor = colunas_numericas[0]  # Usar primeira coluna numérica
    
    if coluna_valor:
        print(f"=== ANÁLISE DE VALORES (usando coluna: {coluna_valor}) ===")
        print(f"Tipo de dados: {df[coluna_valor].dtype}")
        print(f"Valores nulos: {df[coluna_valor].isnull().sum()}")
        print(f"Valor mínimo: {df[coluna_valor].min():,.2f}")
        print(f"Valor máximo: {df[coluna_valor].max():,.2f}")
        print(f"Valor médio: {df[coluna_valor].mean():,.2f}")
        print(f"Soma total: {df[coluna_valor].sum():,.2f}")
        
        # Verificar se há valores negativos ou zero
        valores_negativos = (df[coluna_valor] < 0).sum()
        valores_zero = (df[coluna_valor] == 0).sum()
        
        if valores_negativos > 0:
            print(f"⚠️ ATENÇÃO: {valores_negativos} valores negativos encontrados")
        if valores_zero > 0:
            print(f"⚠️ ATENÇÃO: {valores_zero} valores zerados encontrados")
            
    else:
        print("❌ Não foi possível identificar coluna de valores")
        print("Colunas numéricas disponíveis:", df.select_dtypes(include=[np.number]).columns.tolist())
else:
    print("❌ Não é possível analisar valores - dados não carregados")

## 6. Teste de Consolidação (Exemplo)

Para verificar a lógica de consolidação, simulamos o processo para um cliente específico, somando os valores por classe de ativo.

In [ ]:
if df is not None and coluna_cliente and coluna_classe and coluna_valor:
    print("=== TESTE DE CONSOLIDAÇÃO ===")
    
    # Pegar cliente com carteira mista se disponível, senão qualquer cliente
    if 'clientes_mistas' in locals() and clientes_mistas:
        cliente_teste = clientes_mistas[0]
        print(f"Testando consolidação para cliente MISTO: {cliente_teste}")
    else:
        cliente_teste = df[coluna_cliente].iloc[0]
        print(f"Testando consolidação para cliente: {cliente_teste}")
    
    # Filtrar dados do cliente
    df_cliente = df[df[coluna_cliente] == cliente_teste].copy()
    
    print(f"
Posições originais do cliente ({len(df_cliente)} registros):")
    for _, row in df_cliente.iterrows():
        valor = row[coluna_valor]
        classe = row[coluna_classe] 
        tipo = row.get('tipo_carteira', 'N/A')
        print(f"  {classe} | {valor:,.2f} | {tipo}")
    
    # Simular consolidação simples - somar por classe
    if 'tipo_carteira' in df_cliente.columns:
        print(f"
Consolidação por classe de ativo:")
        consolidado = df_cliente.groupby(coluna_classe)[coluna_valor].sum().sort_values(ascending=False)
        
        total_cliente = consolidado.sum()
        for classe, valor in consolidado.items():
            percentual = (valor / total_cliente) * 100
            print(f"  {classe}: {valor:,.2f} ({percentual:.1f}%)")
        
        print(f"
Total do cliente: {total_cliente:,.2f}")
        
        # Verificar se soma bate com original
        total_original = df_cliente[coluna_valor].sum()
        if abs(total_cliente - total_original) < 0.01:  # tolerância para arredondamentos
            print("✅ Consolidação OK - totais conferem")
        else:
            print(f"⚠️ ATENÇÃO: Diferença nos totais: {abs(total_cliente - total_original):,.2f}")
    else:
        print("⚠️ Não é possível fazer consolidação completa - tipo de carteira não definido")
        
else:
    print("❌ Não é possível testar consolidação - colunas essenciais não identificadas")

## 7. Resumo Final e Insights

Coletamos os principais resultados da análise exploratória, que servirão de base para o desenvolvimento do notebook principal de consolidação.

In [ ]:
print("=" * 70)
print("RESUMO EXECUTIVO - INSIGHTS PARA CONSOLIDAÇÃO")
print("=" * 70)

if df is not None:
    print(f"📊 DADOS CARREGADOS:")
    print(f"   • Total de registros: {len(df):,}")
    print(f"   • Clientes únicos: {df[coluna_cliente].nunique():,}")
    print(f"   • Classes de ativo: {df[coluna_classe].nunique():,}")
    
    if coluna_valor:
        print(f"   • Patrimônio total: {df[coluna_valor].sum():,.2f}")
    
    print(f"
🔍 COLUNAS IDENTIFICADAS:")
    print(f"   • Cliente: {coluna_cliente}")
    print(f"   • Classe de Ativo: {coluna_classe}")
    print(f"   • Valor: {coluna_valor}")
    
    if 'classificacao' in locals():
        print(f"
🏦 CLASSIFICAÇÃO LOCAL/OFFSHORE:")
        tipos_count = {}
        for tipo in classificacao.values():
            tipos_count[tipo] = tipos_count.get(tipo, 0) + 1
        for tipo, count in tipos_count.items():
            print(f"   • {tipo}: {count} classes")
    
    if 'clientes_mistas' in locals():
        print(f"
👥 CLIENTES COM CARTEIRAS MISTAS:")
        print(f"   • Local + Offshore: {len(clientes_mistas):,}")
        print(f"   • Apenas Local: {len(clientes_so_local):,}")
        print(f"   • Apenas Offshore: {len(clientes_so_offshore):,}")
    
    print(f"
⚙️ PARÂMETROS PARA NOTEBOOK PRINCIPAL:")
    print(f"   • COLUNA_CLIENTE = '{coluna_cliente}'")
    print(f"   • COLUNA_CLASSE = '{coluna_classe}'")
    print(f"   • COLUNA_VALOR = '{coluna_valor}'")
    
    if 'termos_local' in locals():
        print(f"   • TERMOS_LOCAL = {termos_local}")
        print(f"   • TERMOS_OFFSHORE = {termos_offshore}")
    
    print(f"
✅ PRÓXIMOS PASSOS:")
    print(f"   1. Usar os parâmetros acima no notebook principal")
    print(f"   2. Implementar lógica de classificação local/offshore")
    print(f"   3. Desenvolver consolidação por cliente + classe")
    print(f"   4. Aplicar regras de recategorização se necessário")
    
else:
    print("❌ FALHA NO CARREGAMENTO - Verificar arquivo e tentar novamente")

print("=" * 70)